# 计算函数的时间和空间消耗

## 准备函数

### 定义一个计算时间-空间的函数

In [1]:
import psutil
import time
import tracemalloc


def compute_peak_memory(func):
    def wrapper(*args, **kwargs):
        tracemalloc.start()

        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()

        _, peak_memory = tracemalloc.get_traced_memory()

        tracemalloc.stop()

        execution_time = end_time - start_time
        print(f"Function {func.__name__} took {execution_time} seconds to execute.")
        print(f"Peak memory usage: {peak_memory} bytes.")
        return execution_time, peak_memory

    return wrapper

In [2]:
# 示例用法
from ripser import ripser
from BOF.get_rank_from_matrix import Effective_Ranks
@compute_peak_memory
def get_TDA(tensor_list:list = []):

    # 现在tensor_list里面有5000个符合高斯分布的张量，每个张量形状为(5000, 32*32*3)
    
    results = []

    for image_matrix in tensor_list:
        TDA = ripser(X=image_matrix, maxdim=1)
        results.append(TDA['dgms'])

    return results

@compute_peak_memory
def get_BOF(tensor_list:list = []):
    # 现在tensor_list里面有5000个符合高斯分布的张量，每个张量形状为(5000, 32*32*3)
        

    results = []
    for image_matrix in tensor_list:
        # 检查张量形状
        if image_matrix.size(0) > image_matrix.size(1):
            image_matrix = image_matrix.T  # 如果 m > n，执行转置操作
        
        get_rank = Effective_Ranks(image_matrix)
        r0 = get_rank.r0
        R0 = get_rank.R0
        rk_max_index = get_rank.rk_max_index
        rk_max = get_rank.rk_max_value
        Rk_max = get_rank.Rk_value_max_rk_index

        results.append({"isic": {"r0": r0, "R0": R0, "rk_max_index": rk_max_index, "rk_max": rk_max, "Rk_max": Rk_max}})
    
    return results

if __name__ == "__main__":
    # my_function()
    import torch
    # 创建一个空列表用于存放张量
    tensor_list = []

    # 定义张量形状
    tensor_shape = (5000, 32*32*3)
    # 生成5000个符合高斯分布的张量并添加到列表中
    for _ in range(10):
        # 使用torch.randn生成符合标准正态分布的张量
        tensor = torch.randn(tensor_shape)
        tensor_list.append(tensor)
        
    TDA_time, TDA_space = get_TDA(tensor_list)
    BOF_time, BOF_space = get_BOF(tensor_list)

    print('时间的比较：{}'.format(TDA_time / BOF_time))
    print('空间比较：{}'.format(TDA_space / BOF_space))


Function get_TDA took 346.20754075050354 seconds to execute.
Peak memory usage: 1306323257 bytes.
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
Function get_BOF took 40.64404559135437 seconds to execute.
Peak memory usage: 274284778 bytes.
时间的比较：8.518038391929846
空间比较：4.762653131994076


# 计算模型参数

## 定义一个计算参数数量的函数

In [14]:
def count_parameters(models):
    parameters_count = []

    for model in models:
        # print(type(model).__name__)
        parameters_count.append(sum(p.numel() for p in model.parameters()))

    return parameters_count

## 计算新ResNet

In [15]:
from nets.resnet import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152
from nets.simple_net import MLP, LeNet

model_list = [MLP(), LeNet(), ResNet18(), ResNet34(), ResNet50(), ResNet101(), ResNet152()]

para_list = count_parameters(model_list)
print(para_list)

[1707274, 136886, 11173962, 21282122, 23520842, 42512970, 58156618]


## 计算旧ResNet

In [16]:
from nets.simple_net import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152
from nets.simple_net import MLP, LeNet

model_list = [MLP(), LeNet(), ResNet18(), ResNet34(), ResNet50(), ResNet101(), ResNet152()]

para_list = count_parameters(model_list)
print(para_list)

[1707274, 136886, 11173962, 21282122, 21282122, 41353546, 57883978]
